In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime
import joblib

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parents[0]))  

from data.db import get_new_connection

apartments = pd.read_sql("SELECT * FROM apartment_listings_processed", get_new_connection())
houses = pd.read_sql("SELECT * FROM house_listings_processed", get_new_connection())

In [ ]:
num_cols_apts = ['log_size', 'rooms', 'size_per_room', 'bathrooms',\
                  'bathrooms_per_room', 'year_built', 'age_of_property']
num_cols_houses = ['log_size', 'log_property_size', 'rooms', 'size_per_room',\
                   'bathrooms', 'bathrooms_per_room', 'year_built', 'age_of_property']

cat_onehot = ['heating']
cat_ordinal_apts = ['condition', 'facade_condition', 'stairwell_condition', 'legal_status']
cat_ordinal_houses = ['condition', 'facade_condition', 'legal_status']

condition_mapping = [['bad', 'average', 'good', 'excellent', 'newly built']]
legal_status_mapping = [['used', 'new']]

ordinal_mappings_apts = [condition_mapping[0], condition_mapping[0], condition_mapping[0], legal_status_mapping[0]]
ordinal_mappings_houses = [condition_mapping[0], condition_mapping[0], legal_status_mapping[0]]

In [ ]:
preprocessor_apts = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), num_cols_apts),
    ('cat_onehot', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), cat_onehot),
    ('cat_ordinal', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal', OrdinalEncoder(categories=ordinal_mappings_apts))
    ]), cat_ordinal_apts)
])

preprocessor_houses = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), num_cols_houses),
    ('cat_onehot', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), cat_onehot),
    ('cat_ordinal', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal', OrdinalEncoder(categories=ordinal_mappings_houses))
    ]), cat_ordinal_houses)
])

In [ ]:
X_apts = apartments.drop(columns=['log_price'])
y_apts = apartments['log_price']

X_houses = houses.drop(columns=['log_price'])
y_houses = houses['log_price']

In [ ]:
X_train_apts, X_test_apts, y_train_apts, y_test_apts = train_test_split(
    X_apts, y_apts, test_size=0.2, random_state=42
)
X_train_houses, X_test_houses, y_train_houses, y_test_houses = train_test_split(
    X_houses, y_houses, test_size=0.2, random_state=42
)

In [ ]:
# XGBoost
if False:
    from xgboost import XGBRegressor
    xgb_params = dict(
        n_estimators = 500,
        learning_rate = 0.05,
        max_depth = 6,
        subsample = 0.8,
        colsample_bytree = 0.8,
        random_state = 42,
        tree_method = 'hist',
    )

    pipeline_apts = Pipeline([
        ('preprocessor', preprocessor_apts),
        ('xgb', XGBRegressor(**xgb_params))
    ])

    pipeline_houses = Pipeline([
        ('preprocessor', preprocessor_houses),
        ('xgb', XGBRegressor(**xgb_params))
    ])

In [ ]:
# XGBoost tuning
if False:
    from xgboost import XGBRegressor
    from sklearn.model_selection import RandomizedSearchCV
    pipeline_apts = Pipeline([
        ('preprocessor', preprocessor_apts),
        ('model', XGBRegressor(tree_method='hist', random_state=42))
    ])

    pipeline_houses = Pipeline([
        ('preprocessor', preprocessor_houses),
        ('model', XGBRegressor(tree_method='hist', random_state=42))
    ])

    # --------------------------
    # 7. Hyperparameter tuning
    # --------------------------
    param_grid = {
        'model__n_estimators': [100, 300, 500],
        'model__max_depth': [3, 5, 6, 8],
        'model__learning_rate': [0.01, 0.05, 0.1],
        'model__subsample': [0.6, 0.8, 1.0],
        'model__colsample_bytree': [0.6, 0.8, 1.0],
        'model__gamma': [0, 0.1, 0.3],
        'model__min_child_weight': [1, 3, 5]
    }

    # Apartments tuning
    rs_apts = RandomizedSearchCV(
        pipeline_apts, param_distributions=param_grid,
        n_iter=50, cv=5, scoring='r2', verbose=2, random_state=42, n_jobs=-1
    )
    rs_apts.fit(X_train_apts, y_train_apts)

    print("Apartments best R^2 (CV):", rs_apts.best_score_)
    print("Apartments best params:", rs_apts.best_params_)

    # Houses tuning
    rs_houses = RandomizedSearchCV(
        pipeline_houses, param_distributions=param_grid,
        n_iter=30, cv=5, scoring='r2', verbose=2, random_state=42, n_jobs=-1
    )
    rs_houses.fit(X_train_houses, y_train_houses)

    print("Houses best R^2 (CV):", rs_houses.best_score_)
    print("Houses best params:", rs_houses.best_params_)

In [ ]:
# Random Forest Regressor
if False:
    from sklearn.ensemble import RandomForestRegressor
    pipeline_apts = Pipeline([
        ('preprocessor', preprocessor_apts),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
    ])

    pipeline_houses = Pipeline([
        ('preprocessor', preprocessor_houses),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
    ])

In [ ]:
# Linear Regression
if False:
    from sklearn.linear_model import LinearRegression
    pipeline_apts = Pipeline([
        ('preprocessor', preprocessor_apts),
        ('model', LinearRegression())
    ])

    pipeline_houses = Pipeline([
        ('preprocessor', preprocessor_houses),
        ('model', LinearRegression())
    ])

In [ ]:
pipeline_apts.fit(X_train_apts, y_train_apts)
pipeline_houses.fit(X_train_houses, y_train_houses)

In [ ]:
# Apartments
y_pred_log_apts = pipeline_apts.predict(X_test_apts)
y_pred_apts = np.exp(y_pred_log_apts)
y_true_apts = np.exp(y_test_apts)

r2_apts = pipeline_apts.score(X_test_apts, y_test_apts)
mae_apts = mean_absolute_error(y_true_apts, y_pred_apts)
rmse_apts = np.sqrt(mean_squared_error(y_true_apts, y_pred_apts))

print("Apartment R^2 (log price): ", r2_apts)
print(f"Apartment MAE (price units): {mae_apts:,.2f}")
print(f"Apartment RMSE (price units): {rmse_apts:,.2f}")

# Houses
y_pred_log_houses = pipeline_houses.predict(X_test_houses)
y_pred_houses = np.exp(y_pred_log_houses)
y_true_houses = np.exp(y_test_houses)

r2_houses = pipeline_houses.score(X_test_houses, y_test_houses)
mae_houses = mean_absolute_error(y_true_houses, y_pred_houses)
rmse_houses = np.sqrt(mean_squared_error(y_true_houses, y_pred_houses))

print("House R^2 (log price): ", r2_houses)
print(f"House MAE (price units): {mae_houses:,.2f}")
print(f"House RMSE (price units): {rmse_houses:,.2f}")

In [ ]:
save = False
if save:
    folder = "RandomForestRegressor"
    os.makedirs(folder, exist_ok=True)
    dt_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    joblib.dump(pipeline_apts, f"{folder}/{dt_str}_apartment_valuation_model.pk1")
    joblib.dump(pipeline_houses, f"{folder}/{dt_str}_house_valuation_model.pk1")

    print(f"Models saved successfully in {folder} directory at {dt_str}.")